## Get the example data at: 
https://doi.org/10.6084/m9.figshare.12646217

In [ ]:
import pandas as pd
from datetime import timedelta

import devicely

In [ ]:
tag_file = 'data/Tags/tags.csv'
empatica_folder = 'data/Empatica'
faros_file = 'data/Faros/faros.EDF'
everion_folder = 'data/Everion'
spacelabs_file = 'data/SpaceLabs/spacelabs.abp'
shimmer_file = 'data/Shimmer/shimmer.csv'

In [ ]:
shift = pd.Timedelta(15,unit='d')

### Read Tags Data

In [ ]:
tags = devicely.TagReader(tag_file)
tags.data.head()

### Timeshift and Write Tags Data

In [ ]:
tags.timeshift(shift)
tags.data.head()

In [ ]:
tags.write('tags.csv')

### Read Bittium Faros 180 Data

In [ ]:
faros = devicely.FarosReader(faros_file)
faros.data.head()

In [ ]:
faros.data['acc_mag'].interpolate(method="time").plot()

### Timeshift and Write Faros Data

In [ ]:
faros.timeshift(shift)
faros.data.head()

In [ ]:
faros.write('faros.csv')

### Read Empatica E4 Data

In [ ]:
empatica = devicely.EmpaticaReader(empatica_folder)

In [ ]:
empatica.sample_freqs

In [ ]:
empatica.start_times

In [ ]:
# empatica.ACC
# empatica.EDA
# empatica.BVP
# empatica.HR
empatica.IBI.head()

In [ ]:
empatica.data.head()

In [ ]:
empatica.data['acc_mag'].interpolate().plot()

### Timeshift and Write Empatica Data

In [ ]:
empatica.timeshift(shift)
empatica.data.head()

In [ ]:
empatica.write('Empatica')

### Read Biovotion Everion Data

In [ ]:
everion = devicely.EverionReader(everion_folder)

In [ ]:
everion.data.head(1)

In [ ]:
everion.data['heart_rate'].plot(style='.')

### Timeshift and Write Everion Data

In [ ]:
everion.timeshift(shift)
everion.data.head()

In [ ]:
everion.write('Everion')

### Read Spacelabs

In [ ]:
spacelabs = devicely.SpacelabsReader(spacelabs_file)
spacelabs.data.head()

In [ ]:
spacelabs.data.plot.scatter('DIA(mmHg)', 'SYS(mmHg)')

### Set Windown and Drop EB on Spacelabs

In [ ]:
spacelabs.set_window(timedelta(seconds=30), 'ffill')

In [ ]:
spacelabs.data.head(1)

In [ ]:
spacelabs.drop_EB()

In [ ]:
spacelabs.data.head(1)

### Timeshift, Deidentify and Write Spacelabs Data

In [ ]:
spacelabs.deidentify('001')
spacelabs.timeshift(shift)

In [ ]:
spacelabs.data.head()

In [ ]:
spacelabs.write('spacelabs.abp')

### Read Shimmer Consensys GSR (Shimmer3 GSR Development Kit)

In [ ]:
shimmer_plus = devicely.ShimmerPlusReader(shimmer_file, delimiter=';')

In [ ]:
shimmer_plus.data.head(1)

In [ ]:
shimmer_plus.data['acc_mag'].interpolate(method="time").plot()

### Timeshift and Write Shimmer Data

In [ ]:
shimmer_plus.timeshift(shift)
shimmer_plus.data.head()

In [ ]:
shimmer_plus.write('shimmer_plus.csv')